In [1]:
import csv
import os
import sys
import pandas as pd
csv.field_size_limit(sys.maxsize)

131072

In [2]:
metrics = ['cbo','wmc','rfc','lcom','nom','nopm','nosm','nof','nopf','nosf','nosi','loc']

dirs = os.listdir("../data/Archive/")
dirs = [d for d in dirs if d != '.DS_Store']

In [3]:
samples =  {'source_code': [],
            'type': [],
            'cbo': [],
            'wmc': [],
            'rfc': [],
            'lcom': [],
            'nom': [],
            'nopm': [],
            'nosm': [],
            'nof': [],
            'nopf': [],
            'nosf': [],
            'nosi': [],
            'loc': [],
            "commits": [],
            "linesAdded": [],
            "linesDeleted": [],
            "authors": [],
            "minorAuthors": [],
            "majorAuthors": [],
            "authorOwnership": [],
            'target': []
            }

metrics2 = ["commits", "linesAdded", "linesDeleted", "authors", "minorAuthors", "majorAuthors", "authorOwnership"]
def get_values(row):
    metrics_values = {}
    for m in metrics2:
        metrics_values[m] = row[m]
    return metrics_values

for d in dirs:
    look_up = {}
    with open('../data/Archive/{}/process.csv'.format(d), 'r') as pro_file:
        pro_csv = csv.DictReader(pro_file, delimiter=';')
        for row in pro_csv:
            look_up["{}_{}".format(row["commit"], row["file"])] = get_values(row)

    with open('../data/Archive/{}/yes-refactoring.csv'.format(d), 'r') as pos_file:
        pos_csv = csv.DictReader(pos_file, delimiter=';')
        for row in pos_csv:
            samples['source_code'].append("../data/Archive/{}/storage/{}/before/{}".format(d,row['before'],row['path']))
            samples['type'].append(row['type'])
            samples['target'].append(row['refactoring'])
            for m in metrics:
                samples[m].append(row[m])
                
            metrics2 = look_up["{}_{}".format(row["before"], row["path"])]
            for k, v in metrics2.items():
                samples[k].append(v)
            
    with open('../data/Archive/{}/no-refactoring.csv'.format(d), 'r') as neg_file:
        neg_csv = csv.DictReader(neg_file, delimiter=';')
        for row in neg_csv:
            samples['source_code'].append("../data/Archive/{}/storage/{}/not-refactored/{}".format(d,row['commit'],row['path']))
            samples['type'].append(row['type'])
            samples['target'].append("NO_REFACTORING")
            for m in metrics:
                samples[m].append(row[m])
                
            metrics2 = look_up["{}_{}".format(row["commit"], row["path"])]
            for k, v in metrics2.items():
                samples[k].append(v)

In [4]:
df = pd.DataFrame.from_dict(samples)
df.head()

,source_code,type,cbo,wmc,rfc,lcom,nom,nopm,nosm,nof,...,nosi,loc,commits,linesAdded,linesDeleted,authors,minorAuthors,majorAuthors,authorOwnership,target
0,../data/Archive/fse19-2/storage/f90d29262b1066...,class,14,4,9,0,3,2,0,2,...,0,91,3,208,0,2,0,2,0.6666666666666666,EXTRACT_VARIABLE
1,../data/Archive/fse19-2/storage/f90d29262b1066...,class,16,5,9,0,3,2,0,2,...,0,94,3,214,0,2,0,2,0.6666666666666666,EXTRACT_VARIABLE
2,../data/Archive/fse19-2/storage/b295635a87da0c...,class,30,74,64,1106,45,36,0,2,...,0,1347,22,146,31,7,2,5,0.36363636363636365,RENAME_METHOD
3,../data/Archive/fse19-2/storage/b295635a87da0c...,class,30,74,64,1106,45,36,0,2,...,0,1347,22,146,31,7,2,5,0.36363636363636365,RENAME_PARAMETER
4,../data/Archive/fse19-2/storage/b295635a87da0c...,class,30,74,64,1106,45,36,0,2,...,0,1347,22,146,31,7,2,5,0.36363636363636365,EXTRACT_AND_MOVE_OPERATION


In [5]:
df.to_pickle('../data/instances.pkl')